In [5]:
import tiktoken
import torch

In [9]:
with open('../data/the-verdict.txt', 'r') as f:
    raw_text = f.read()
print('total chars:', len(raw_text))
print(raw_text[:50])

total chars: 20479
I HAD always thought Jack Gisburn rather a cheap g


In [16]:
import re

text = 'Hello, world. This, is a test.'
result = re.split(r'([,.]|\s)', text)
result = [ele.strip() for ele in result if ele.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [22]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!()\']|--|\s)', text)
result = [ele.strip() for ele in result if ele.strip()]
result

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']

In [25]:
preprocessed = re.split(r'([,.:;?"_!()\']|--|\s)', raw_text)
preprocessed = [ele.strip() for ele in preprocessed if ele.strip()]
print(preprocessed[:10])
print('total tokens', len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']
total tokens 4690


In [26]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print('vocab size', vocab_size)

vocab size 1130


In [28]:
vocab = {token:i for i,token in enumerate(all_words)}
for token, i in vocab.items():
    print(i, '->', token)
    if i >= 20:
        break

0 -> !
1 -> "
2 -> '
3 -> (
4 -> )
5 -> ,
6 -> --
7 -> .
8 -> :
9 -> ;
10 -> ?
11 -> A
12 -> Ah
13 -> Among
14 -> And
15 -> Are
16 -> Arrt
17 -> As
18 -> At
19 -> Be
20 -> Begin


In [55]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_i = vocab
        self.i_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, raw_text):
        preprocessed = re.split(r'([,.:;?"_!()\']|--|\s)', raw_text)
        preprocessed = [ele.strip() for ele in preprocessed if ele.strip()]
        ids = [self.str_to_i[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        tokens = [self.i_to_str[id] for id in ids]
        decoded_text =  " ".join(tokens)
        decoded_text = re.sub(r'\s+([,.?!"()\'])', r'\1', decoded_text)
        return decoded_text    

In [56]:
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
sm_tokenizer = SimpleTokenizerV1(vocab)
ids = sm_tokenizer.encode(text)
print('ids', ids[:5])
decoded_text = sm_tokenizer.decode(ids)
print('decoded text', decoded_text)

ids [1, 56, 2, 850, 988]
decoded text " It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [58]:
text = "Hello, do you like tea. Is this-- a test?"
sm_tokenizer = SimpleTokenizerV1(vocab)
sm_tokenizer.encode(text)

KeyError: 'Hello'

In [74]:
preprocessed = re.split(r'([,.:;?"_!()\']|--|\s)', raw_text)
preprocessed = [ele.strip() for ele in preprocessed if ele.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {s:i for i,s in enumerate(all_tokens)}
print(len(vocab))

1132


In [75]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_i = vocab
        self.i_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, raw_text):
        preprocessed = re.split(r'([,.:;?"_!()\']|--|\s)', raw_text)
        preprocessed = [ele.strip() for ele in preprocessed if ele.strip()]
        preprocessed = [ele if ele in self.str_to_i else '<|unk|>' for ele in preprocessed]
        ids = [self.str_to_i[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        tokens = [self.i_to_str[id] for id in ids]
        decoded_text =  " ".join(tokens)
        decoded_text = re.sub(r'\s+([,.?!"()\'])', r'\1', decoded_text)
        return decoded_text    

In [76]:
text = "Hello, do you like tea. Is this-- a test?"
sm_tokenizer = SimpleTokenizerV2(vocab)
ids = sm_tokenizer.encode(text)
print('ids', ids)
print('decoded texts', sm_tokenizer.decode(ids))

ids [1131, 5, 355, 1126, 628, 975, 7, 1131, 999, 6, 115, 1131, 10]
decoded texts <|unk|>, do you like tea. <|unk|> this -- a <|unk|>?


In [79]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
sm_tokenizer = SimpleTokenizerV2(vocab)
text = " <|endoftext|> ".join([text1, text2])
print('text', text)
ids = sm_tokenizer.encode(text)
print('ids', ids)
print('decode text', sm_tokenizer.decode(ids))

text Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
ids [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
decode text <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [85]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
gpt2_tokenizer = tiktoken.get_encoding('gpt2')
ids = gpt2_tokenizer.encode(text, allowed_special={'<|endoftext|>'})
print('ids', ids)
print('decode text', gpt2_tokenizer.decode(ids))

ids [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
decode text Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [86]:
enc_text = gpt2_tokenizer.encode(raw_text)
print('tokens', len(enc_text))

tokens 5145


In [88]:
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print('x', x)
print('y', y)

x [290, 4920, 2241, 287]
y [4920, 2241, 287, 257]


In [89]:
for i in range(1, context_size+1):
    print(enc_sample[:i], '->', enc_sample[i])

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [93]:
for i in range(1, context_size+1):
    print(gpt2_tokenizer.decode(enc_sample[:i]), '->', gpt2_tokenizer.decode([enc_sample[i]]))

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


In [94]:
import torch
from torch.utils.data import DataLoader, Dataset

In [121]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []
        
        tokens = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
        
        for i in range(0, len(tokens)-max_len, stride):
            input_chunk = tokens[i:i+max_len]
            target_chunk = tokens[i+1:i+max_len+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [122]:
def create_dataloaderv1(text, batch_size=4, max_len=256, stride=128,
                        drop_last=True, shuffle=True, num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(text, tokenizer, max_len, stride)
    return DataLoader(
        dataset=dataset,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        batch_size=batch_size
    )

In [124]:
dataloder = create_dataloaderv1(raw_text, batch_size=1, stride=1, max_len=4, shuffle=False)
data_iter = iter(dataloder)
batch_1 = next(data_iter)
print(batch_1)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [125]:
batch_2 = next(data_iter)
print(batch_2)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [126]:
dataloder = create_dataloaderv1(raw_text, batch_size=8, max_len=4, stride=4, shuffle=False)
data_iter = iter(dataloder)
batch_1 = next(data_iter)
print(batch_1)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [127]:
input_ids = torch.tensor([1, 2, 3, 5])
vocab_size = 6
output_dim = 3
embeddding_layer = torch.nn.Embedding(vocab_size, output_dim)
embeddding_layer(input_ids)

tensor([[ 0.8261,  0.9677,  0.4608],
        [ 2.3274, -0.7171, -0.8277],
        [-0.1206,  0.1012, -1.3381],
        [ 1.4303, -0.6018, -0.8481]], grad_fn=<EmbeddingBackward0>)

In [128]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [131]:
dataloder = create_dataloaderv1(raw_text, batch_size=8, max_len=4, stride=4, shuffle=False)
data_iter = iter(dataloder)
batch_1 = next(data_iter)
print(batch_1[0].shape)

torch.Size([8, 4])


In [138]:
text_embedding = token_embedding_layer(batch_1[0])
text_embedding.shape

torch.Size([8, 4, 256])

In [139]:
context_len = 4
pos_embedding_layer = torch.nn.Embedding(context_len, output_dim)

In [140]:
pos_embedding = pos_embedding_layer(torch.arange(context_len))
pos_embedding.shape

torch.Size([4, 256])

In [141]:
input_embedding = text_embedding + pos_embedding
input_embedding.shape

torch.Size([8, 4, 256])